In [3]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.model_selection import train_test_split
import torch

# Load the Romanian text summarization dataset
data = load_dataset("readerbench/ro-text-summarization")

# Prepare and split the dataset
train_val_split = data["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = train_val_split["train"]
validation_dataset = train_val_split["test"]

# Load tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Function to tokenize the inputs and labels
def preprocess_function(examples):
    inputs = [doc for doc in examples["Content"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Summary"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to training and validation data
train_dataset = train_dataset.map(preprocess_function, batched=True)
validation_dataset = validation_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # Output directory for model checkpoints
    evaluation_strategy='epoch',     # Evaluation is done at the end of each epoch
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=4,   # Training batch size
    per_device_eval_batch_size=4,    # Evaluation batch size
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Weight decay
    save_total_limit=3,              # Only save the last three checkpoints
    predict_with_generate=True       # Use model.generate for predictions
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()


# Load the ROUGE metric to evaluate performance
from datasets import load_metric
rouge_metric = load_metric("rouge")

# Get predictions for validation dataset
val_predictions = trainer.predict(validation_dataset)

# Compute ROUGE scores
rouge_scores = rouge_metric.compute(
    predictions=[tokenizer.decode(output, skip_special_tokens=True) for output in val_predictions.predictions],
    references=[item["summary"] for item in validation_dataset]
)

print("ROUGE scores:", rouge_scores)

# Test the model with a sample input
sample_text = """Buzz House este un film care va fi lansat pe 23 aprilie 2024. Este o producție a studiourilor "Star Films", cunoscut pentru filmele sale cu teme inovatoare și regizori premiati. Filmul urmărește viața a patru personaje care locuiesc într-o casă zgomotoasă și plină de aventuri neprevăzute. Fiecare cameră a casei are o poveste diferită, cu decoruri bogate și detalii atent alese. Distributia include actori cunoscuți din industria de teatru și film, precum Ion Popescu, care a jucat în numeroase filme premiate la festivaluri internaționale.

Maria Ionescu, de asemenea, face parte din distribuție și este cunoscută pentru rolurile sale în serialele de televiziune. Andrei Vasilescu, care a studiat la o școală de teatru celebră în New York, aduce o energie specială filmului. Elena Georgescu, ultimul nume important din distribuție, este actriță și cântăreață, cunoscută pentru concertele ei pline de viață.

Regizorul filmului este recunoscut pentru stilul său unic, care îmbină comedia cu drama și elemente de film noir. În timpul filmărilor, a fost un entuziasm mare pe platou, cu toți actorii implicându-se în improvizații pentru a adăuga profunzime personajelor. Povestea filmului este plină de surprize, iar spectatorii se pot aștepta la râsete și lacrimi deopotrivă. În plus, coloana sonoră este creată de un compozitor premiat, ceea ce adaugă un element suplimentar de emoție.

Casa în care s-a filmat este o proprietate veche, cu o istorie interesantă, și a fost aleasă pentru atmosfera sa unică. Regizorul a declarat că s-a inspirat din experiențe reale pentru a scrie scenariul. De asemenea, s-au folosit multe costume autentice din perioada filmului pentru a conferi autenticitate. Spectatorii pot aștepta scene memorabile și un final care îi va face să reflecteze asupra vieții lor. Filmul promite să fie unul dintre cele mai interesante ale anului.
"""

# Tokenize and generate a summary
inputs = tokenizer(sample_text, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs["input_ids"], max_length=100, min_length=20, do_sample=False)

summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Sample text:", sample_text)
print("Generated summary:", summary)


c:\Users\stoic\miniconda3\envs\myenv\lib\site-packages\transformers\tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(





















































Map: 100%|██████████| 52214/52214 [00:39<00:00, 1320.16 examples/s]














Map: 100%|██████████| 13054/13054 [00:08<00:00, 1461.80 examples/s]












































